# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [20]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.conda_dependencies import CondaDependencies


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [21]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-run'

experiment=Experiment(ws, experiment_name)

In [22]:
# Load the registered dataset from workspace
dataset = Dataset.get_by_name(ws, name='diabetes')

# Convert the dataset to dataframe
df = dataset.to_pandas_dataframe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [23]:
amlcompute_cluster_name = "cluster"

# Verify if cluster does not exist otherwise use the existing one
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',
                                                           vm_priority = 'lowpriority', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [24]:
# create environment
environment = Environment(name="azure-env")
conda_env = CondaDependencies()

# Needed packages and scripts
conda_env.add_conda_package("pandas")
conda_env.add_conda_package("numpy")
conda_env.add_conda_package("scikit-learn")
conda_env.add_pip_package("azureml-defaults")

# Adding dependencies to the created environment
environment.python.conda_dependencies=conda_env

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [48]:
# Create an early termination policy
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

# Create different params that will be used during training
param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.001, 0.01, 0.1),
        '--max_iter': choice(1, 2)
    }
)

# Create estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory = ".",
                            script='train.py',
                            arguments=['--data', dataset.id],
                            compute_target=amlcompute_cluster_name,
                            environment=environment)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=24,
                                     max_concurrent_runs=4)

In [49]:
# Submit experiment
run = experiment.submit(hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [50]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [51]:
run.wait_for_completion(show_output=True)

RunId: HD_778f89d5-b3b4-4e72-8997-128fcfeee96e
Web View: https://ml.azure.com/runs/HD_778f89d5-b3b4-4e72-8997-128fcfeee96e?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-222426/workspaces/quick-starts-ws-222426&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-01-13T14:35:53.979508][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-01-13T14:35:55.7244947Z][SCHEDULER][INFO]Scheduling job, id='HD_778f89d5-b3b4-4e72-8997-128fcfeee96e_0' 
[2023-01-13T14:35:55.9260259Z][SCHEDULER][INFO]Scheduling job, id='HD_778f89d5-b3b4-4e72-8997-128fcfeee96e_1' 
[2023-01-13T14:35:56.0977951Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_778f89d5-b3b4-4e72-8997-128fcfeee96e_0' 
[2023-01-13T14:35:56.1332176Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_778f89d5-b3b4-4e72-8997-128fcfeee96e_1' 
[2023-01-13T14:35:56.132181][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be 

{'runId': 'HD_778f89d5-b3b4-4e72-8997-128fcfeee96e',
 'target': 'cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-01-13T14:35:53.208303Z',
 'endTimeUtc': '2023-01-13T14:39:27.214908Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '7cd819ed-31db-43d4-85c1-8aa173e0766e',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': '6',
  'score': '0.7604166666666666',
  'best_child_run_id': 'HD_778f89d5-b3b4-4e72-8997-128fcfeee96e_3',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_778f89d5-b3b4-4e72-8997-128fcfeee96e_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType':

In [52]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive-run,HD_778f89d5-b3b4-4e72-8997-128fcfeee96e,hyperdrive,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [30]:
# Retrieve and save best model
best_run = run.get_best_run_by_primary_metric()
best_run.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2023-01-13-13',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [39]:
# Registring the best model
model = best_run.register_model(model_name='hd-best-model', model_path='outputs/model.joblib')

In [40]:
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-222426', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-222426'), name=hd-best-model, id=hd-best-model:6, version=6, tags={}, properties={})


In [33]:
inference_config = InferenceConfig(entry_script='score.py',
                                   environment=environment)
service_name = 'hd-deploy-1'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True
                      )
service.wait_for_deployment(show_output=True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoint-sdk-v2 /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-01-13 14:12:34+00:00 Creating Container Registry if not exists.
2023-01-13 14:12:35+00:00 Registering the environment.
2023-01-13 14:12:35+00:00 Use the existing image.
2023-01-13 14:12:36+00:00 Generating deployment configuration.
2023-01-13 14:12:36+00:00 Submitting deployment to compute.
2023-01-13 14:12:38+00:00 Checking the status of deployment hd-deploy-1..
2023-01-13 14:16:21+00:00 Checking the status of inference endpoint hd-deploy-1.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [34]:
# Enable app insights
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [16]:
data = [{"Pregnancies": 6, 
 "Glucose": 148, 
 "BloodPressure": 72, 
 "SkinThickness": 35, 
 "Insulin": 0, 
 "BMI": 33.5, 
 "DiabetesPedigreeFunction": 0.627, 
 "Age": 50},
        
{"Pregnancies": 1, 
 "Glucose": 85, 
 "BloodPressure": 66, 
 "SkinThickness": 29, 
 "Insulin": 20, 
 "BMI": 26.5, 
 "DiabetesPedigreeFunction": 0.351, 
 "Age": 31}]

print(data)

[{'Pregnancies': 6, 'Glucose': 148, 'BloodPressure': 72, 'SkinThickness': 35, 'Insulin': 0, 'BMI': 33.5, 'DiabetesPedigreeFunction': 0.627, 'Age': 50}, {'Pregnancies': 1, 'Glucose': 85, 'BloodPressure': 66, 'SkinThickness': 29, 'Insulin': 20, 'BMI': 26.5, 'DiabetesPedigreeFunction': 0.351, 'Age': 31}]


TODO: In the cell below, print the logs of the web service and delete the service

In [36]:
import requests
import json

scoring_uri = service.scoring_uri
#two set of data to score, so we get two results back
data = {"data": [{"Pregnancies": 5, 
     "Glucose": 150, 
     "BloodPressure": 70, 
     "SkinThickness": 40, 
     "Insulin": 10, 
     "BMI": 36.5, 
     "DiabetesPedigreeFunction": 0.627, 
     "Age": 30},

    {"Pregnancies": 5, 
     "Glucose": 90, 
     "BloodPressure": 70, 
     "SkinThickness": 34, 
     "Insulin": 20, 
     "BMI": 26.5, 
     "DiabetesPedigreeFunction": 0.351, 
     "Age": 28},
      ]}
    
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())
print("Case 0: Not Diabetes, Case 1: Diabetes.")

name 'model' is not defined
Case 0: Not Diabetes, Case 1: Diabetes.


In [17]:
import json


input_payload = json.dumps({
    'data': df[0:2].values.tolist(),
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

name 'model' is not defined


In [18]:
logs = service.get_logs()
logs

'/bin/bash: /azureml-envs/azureml_e11055d273150b599f42d9b7de24acbb/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_e11055d273150b599f42d9b7de24acbb/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_e11055d273150b599f42d9b7de24acbb/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n/bin/bash: /azureml-envs/azureml_e11055d273150b599f42d9b7de24acbb/lib/libtinfo.so.6: no version information available (required by /bin/bash)\n2023-01-13T13:43:02,855547667+00:00 - iot-server/run \n2023-01-13T13:43:02,856138264+00:00 - rsyslog/run \n2023-01-13T13:43:02,866607013+00:00 - gunicorn/run \n2023-01-13T13:43:02,874050276+00:00 | gunicorn/run | \n2023-01-13T13:43:02,876088266+00:00 | gunicorn/run | ###############################################\nbash: /azureml-envs/azureml_e11055d273150b599f42d9b7de24acbb/lib/libtinfo.so.6: no version information availab

In [19]:
service.delete()

WebserviceException: WebserviceException:
	Message: There is a deployment operation in flight for the Service: hd-deploy-1
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "There is a deployment operation in flight for the Service: hd-deploy-1"
    }
}